In [5]:
from sagas.ofbiz.entity_gen import gen_bloc_model
lines = []
# entities = ['Testing', 'TestingType', 'TestFieldType', 'Person']
entities = ['Person']
for entity_name in entities:
    # entity_name='TestFieldType'
    gen_bloc_model(lines, entity_name)
print('\n'.join(lines))


/// Entity Person, Person
class Person extends EntityBase {

  /// this entity has only one pk
  final int yearsWithEmployer;
  final String lastName;
  final String occupation;
  final String gender;
  final String employmentStatusEnumId;
  final String socialSecurityNumber;
  final String suffix;
  final String mothersMaidenName;
  final String middleNameLocal;
  final String existingCustomer;
  final String residenceStatusEnumId;
  final String nickname;
  final String partyId; // pk
  final String memberId;
  final double height;
  final String passportNumber;
  final String lastNameLocal;
  final String comments;
  final int monthsWithEmployer;
  final double weight;
  final DateTime birthDate;
  final String otherLocal;
  final String firstName;
  final String cardId;
  final String middleName;
  final String firstNameLocal;
  final DateTime passportExpireDate;
  final String salutation;
  final String personalTitle;
  final DateTime deceasedDate;
  final double totalYearsWorkEx

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker

engine = create_engine('sqlite:///database.sqlite3', convert_unicode=True)
db_session = scoped_session(sessionmaker(autocommit=False,
                                         autoflush=False,
                                         bind=engine))
Base = declarative_base()
Base.query = db_session.query_property()

In [3]:
# from database import Base
from sqlalchemy import Column, DateTime, ForeignKey, Integer, String, func
from sqlalchemy.orm import backref, relationship


class Department(Base):
    __tablename__ = 'department'
    id = Column(Integer, primary_key=True)
    name = Column(String)


class Role(Base):
    __tablename__ = 'roles'
    role_id = Column(Integer, primary_key=True)
    name = Column(String)


class Employee(Base):
    __tablename__ = 'employee'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    # Use default=func.now() to set the default hiring time
    # of an Employee to be the current time when an
    # Employee record was created
    hired_on = Column(DateTime, default=func.now())
    department_id = Column(Integer, ForeignKey('department.id'))
    role_id = Column(Integer, ForeignKey('roles.role_id'))
    # Use cascade='delete,all' to propagate the deletion of a Department onto its Employees
    department = relationship(
        Department,
        backref=backref('employees',
                        uselist=True,
                        cascade='delete,all'))
    role = relationship(
        Role,
        backref=backref('roles',
                        uselist=True,
                        cascade='delete,all'))

In [4]:
def init_db():
    # import all modules here that might define models so that
    # they will be registered properly on the metadata.  Otherwise
    # you will have to import them first before calling init_db()
    # from models import Department, Employee, Role
    Base.metadata.drop_all(bind=engine)
    Base.metadata.create_all(bind=engine)

    # Create the fixtures
    engineering = Department(name='Engineering')
    db_session.add(engineering)
    hr = Department(name='Human Resources')
    db_session.add(hr)

    manager = Role(name='manager')
    db_session.add(manager)
    engineer = Role(name='engineer')
    db_session.add(engineer)

    peter = Employee(name='Peter', department=engineering, role=engineer)
    db_session.add(peter)
    roy = Employee(name='Roy', department=engineering, role=engineer)
    db_session.add(roy)
    tracy = Employee(name='Tracy', department=hr, role=manager)
    db_session.add(tracy)
    db_session.commit()
    
init_db()    

In [5]:
DepartmentModel=Department
EmployeeModel=Employee
RoleModel=Role

In [6]:
import graphene
from graphene import relay
from graphene_sqlalchemy import SQLAlchemyConnectionField, SQLAlchemyObjectType


class Department(SQLAlchemyObjectType):
    class Meta:
        model = DepartmentModel
        interfaces = (relay.Node, )


class Employee(SQLAlchemyObjectType):
    class Meta:
        model = EmployeeModel
        interfaces = (relay.Node, )


class Role(SQLAlchemyObjectType):
    class Meta:
        model = RoleModel
        interfaces = (relay.Node, )


class Query(graphene.ObjectType):
    node = relay.Node.Field()
    # Allow only single column sorting
    all_employees = SQLAlchemyConnectionField(
        Employee, sort=Employee.sort_argument())
    # Allows sorting over multiple columns, by default over the primary key
    all_roles = SQLAlchemyConnectionField(Role)
    # Disable sorting over this field
    all_departments = SQLAlchemyConnectionField(Department, sort=None)


schema = graphene.Schema(query=Query, types=[Department, Employee, Role])

In [12]:
def to_std_dicts(value):
    """Convert nested ordered dicts to normal dicts for better comparison."""
    if isinstance(value, dict):
        return {k: to_std_dicts(v) for k, v in value.items()}
    elif isinstance(value, list):
        return [to_std_dicts(v) for v in value]
    else:
        return value
    
query = """
{
  allEmployees(sort: [NAME_ASC, ID_ASC]) {
    edges {
      node {
        id
        name
        department {
          id
          name
        }
        role {
          id
          name
        }
      }
    }
  }
}
"""
result = schema.execute(query, context_value={'session': db_session})
print(result.errors)
to_std_dicts(result.data)

None


{'allEmployees': {'edges': [{'node': {'id': 'RW1wbG95ZWU6MQ==',
     'name': 'Peter',
     'department': {'id': 'RGVwYXJ0bWVudDox', 'name': 'Engineering'},
     'role': {'id': 'Um9sZToy', 'name': 'engineer'}}},
   {'node': {'id': 'RW1wbG95ZWU6Mg==',
     'name': 'Roy',
     'department': {'id': 'RGVwYXJ0bWVudDox', 'name': 'Engineering'},
     'role': {'id': 'Um9sZToy', 'name': 'engineer'}}},
   {'node': {'id': 'RW1wbG95ZWU6Mw==',
     'name': 'Tracy',
     'department': {'id': 'RGVwYXJ0bWVudDoy', 'name': 'Human Resources'},
     'role': {'id': 'Um9sZTox', 'name': 'manager'}}}]}}

In [16]:
# query = """
#     query {
#       allDepartments {
#         id
#         name
#       }      
#     }
# """
# result = schema.execute(query, context_value={'session': db_session})
# print(result.errors)
# to_std_dicts(result.data)

[GraphQLError('Cannot query field "id" on type "DepartmentConnection".',), GraphQLError('Cannot query field "name" on type "DepartmentConnection".',)]


In [24]:
import enum
from sqlalchemy import (Column, Date, Enum, ForeignKey, Integer, String, Table,
                        func, select)
from sqlalchemy.ext.hybrid import hybrid_property

PetKind = Enum("cat", "dog", name="pet_kind")
association_table = Table(
    "association",
    Base.metadata,
    Column("pet_id", Integer, ForeignKey("pets.id")),
    Column("reporter_id", Integer, ForeignKey("reporters.id")),
)

class Reporter(Base):
    __tablename__ = "reporters"

    id = Column(Integer(), primary_key=True)
    first_name = Column(String(30), doc="First name")
    last_name = Column(String(30), doc="Last name")
    email = Column(String(), doc="Email")
    favorite_pet_kind = Column(PetKind)
    pets = relationship("Pet", secondary=association_table, backref="reporters")
    articles = relationship("Article", backref="reporter")
    favorite_article = relationship("Article", uselist=False)

    @hybrid_property
    def hybrid_prop(self):
        return self.first_name

    column_prop = column_property(
        select([func.cast(func.count(id), Integer)]), doc="Column property"
    )

    composite_prop = composite(CompositeFullName, first_name, last_name, doc="Composite")

class Article(Base):
    __tablename__ = "articles"
    id = Column(Integer(), primary_key=True)
    headline = Column(String(100))
    pub_date = Column(Date())
    reporter_id = Column(Integer(), ForeignKey("reporters.id"))


InvalidRequestError: Table 'association' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.